In [12]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

#Cleaning text
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Temor AL-
[nltk_data]     Kaisi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[nltk_data] Downloading package punkt to C:\Users\Temor AL-
[nltk_data]     Kaisi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
items = pd.read_csv("20191226-items.csv")
reviews = pd.read_csv("20191226-reviews.csv")

reviews['body'][0]
type(reviews)

pandas.core.frame.DataFrame

In [14]:
# 0.4 Link review data to item data bsed on common column "asin"
reviews = pd.merge(reviews, items, how="left", left_on="asin", right_on="asin")
reviews.head()
print(reviews)

             asin               name  rating_x                date  verified  \
0      B0000SX2UC              Janet         3    October 11, 2005     False   
1      B0000SX2UC         Luke Wyatt         1     January 7, 2004     False   
2      B0000SX2UC             Brooke         5   December 30, 2003     False   
3      B0000SX2UC      amy m. teague         3      March 18, 2004     False   
4      B0000SX2UC      tristazbimmer         4     August 28, 2005     False   
...           ...                ...       ...                 ...       ...   
67981  B081H6STQQ              jande         5     August 16, 2019     False   
67982  B081H6STQQ            2cool4u         5  September 14, 2019     False   
67983  B081H6STQQ              simon         5       July 14, 2019     False   
67984  B081TJFVCJ  Tobiasz Jedrysiak         5   December 24, 2019      True   
67985  B0825BB7SG      Owen Gonzalez         5   December 11, 2019     False   

                                       

In [15]:
# 0.5 Rename columns
reviews.rename(columns={"rating_x": "rating", "title_x": "title", "title_y": "item_title", "rating_y": "overall_rating"}, inplace=True)
#reviews["id"] = reviews.index + 1
reviews.head(15)

,asin,name,rating,date,verified,title,body,helpfulVotes,brand,item_title,url,image,overall_rating,reviewUrl,totalReviews,price,originalPrice
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,3.0,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.00,0.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,3.0,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.00,0.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,3.0,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.00,0.0
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,3.0,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.00,0.0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,3.0,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.00,0.0
5,B0000SX2UC,J. White,4,"September 25, 2005",False,Worked great for me,"Hello, I have this phone and used it until I d...",NaN,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,3.0,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.00,0.0
6,B0000SX2UC,the cell phone store owner,5,"April 16, 2004",False,Wanna cool Nokia? You have it here!,Cool. Cheap. Color: 3 words that describe the ...,2.0,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,3.0,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.00,0.0
7,B0000SX2UC,Matt,4,"April 3, 2004",False,Problem with 3588i universal headset,"The 3599i is overall a nice phone, except that...",2.0,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,3.0,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.00,0.0
8,B0000SX2UC,Charles Cook,5,"November 24, 2003",False,cool phone!!!!!!!!,"I've never owned a Nokia phone before, so this...",7.0,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,3.0,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.00,0.0
9,B0000SX2UC,Amazon Customer,3,"February 2, 2004",False,Pissed off-a little bit,ok well im in school and i need the text messa...,3.0,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,3.0,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.00,0.0


In [16]:
reviews# Remove null values and unneeded features
reviews = reviews.dropna()
reviews = reviews.reset_index(drop=True)

In [17]:
# 2.2 Create brand subsets
apple = reviews[reviews["brand"]=="Apple"].sort_values(by=["date"], ascending=False)

In [18]:
apple['body'] = apple['body'].str.lower()

In [19]:
def tokenization_s(sentences): # same can be achieved for words tokens
    s_new = []
    for sent in (sentences[:][0]): #For NumpY = sentences[:]
        s_token = sent_tokenize(sent)
        if s_token != '':
            s_new.append(s_token)
    return s_new

In [20]:
apple["clean_text"] = apple["body"].apply(tokenization_s)

In [21]:
apple

,asin,name,rating,date,verified,title,body,helpfulVotes,brand,item_title,url,image,overall_rating,reviewUrl,totalReviews,price,originalPrice,clean_text
17809,B079HB518K,Corey,5,"September 9, 2018",True,Satisfied customer,phone came with no sign of wear in great condi...,14.0,Apple,"Apple iPhone 7, 32GB, Rose Gold - For AT&T / T...",https://www.amazon.com/Apple-iPhone-GSM-Unlock...,https://m.media-amazon.com/images/I/71x3e0x+M2...,3.9,https://www.amazon.com/product-reviews/B079HB518K,243,199.00,206.90,[[p]]
16854,B077J7H62P,Andy Nhat Tran,2,"September 8, 2019",True,Battery life is so bad,i don’t want to buy it any more,3.0,Apple,Apple MGLW2LL/A iPad Air 2 9.7-Inch Retina Dis...,https://www.amazon.com/Apple-MGLW2LL-9-7-Inch-...,https://m.media-amazon.com/images/I/61ufQeEma5...,4.2,https://www.amazon.com/product-reviews/B077J7H62P,124,211.39,0.00,[[i]]
12608,B06XS3CTJM,L.C.,5,"September 8, 2019",True,"Great product, like new!",i love it! works like new and exactly as descr...,1.0,Apple,Apple iPhone 7 Plus 256GB Unlocked GSM 4G LTE ...,https://www.amazon.com/Apple-iPhone-Unlocked-Q...,https://m.media-amazon.com/images/I/71c5RZEpLw...,4.0,https://www.amazon.com/product-reviews/B06XS3CTJM,193,354.99,0.00,[[i]]
12606,B06XS3CTJM,Amazon Customer,5,"September 8, 2019",True,Looks and works brand new,"works perfect, easy to use with my cricket sim...",2.0,Apple,Apple iPhone 7 Plus 256GB Unlocked GSM 4G LTE ...,https://www.amazon.com/Apple-iPhone-Unlocked-Q...,https://m.media-amazon.com/images/I/71c5RZEpLw...,4.0,https://www.amazon.com/product-reviews/B06XS3CTJM,193,354.99,0.00,[[w]]
16598,B07755LZ67,Talha Farooq,5,"September 8, 2018",True,Excellent Product,"absolutely new, just no earphones came in the box",16.0,Apple,Apple iPhone 8 Plus 64GB Unlocked GSM Phone - ...,https://www.amazon.com/Apple-iPhone-Unlocked-G...,https://m.media-amazon.com/images/I/810MbmOEoq...,4.0,https://www.amazon.com/product-reviews/B07755LZ67,176,374.99,0.00,[[a]]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12382,B06XRN4RWL,Amazon Customer,3,"April 10, 2019",True,Some Issues,i ordered the iphone 7 plus 128gb gold and got...,4.0,Apple,"Apple iPhone 7 Plus, 128GB, Gold - For Sprint ...",https://www.amazon.com/Apple-iPhone-128GB-Unlo...,https://m.media-amazon.com/images/I/71NwKwSqSA...,3.8,https://www.amazon.com/product-reviews/B06XRN4RWL,28,304.99,0.00,[[i]]
11571,B01N9YO1DS,Peter Van Dijck,1,"April 10, 2018",True,iPhone is NOT unlocked.,iphone is wasn't unlocked.,1.0,Apple,"Apple iPhone 7, 128GB, Gold - For AT&T / T-Mob...",https://www.amazon.com/Apple-iPhone-GSM-Unlock...,https://m.media-amazon.com/images/I/71RYhD1uzp...,3.5,https://www.amazon.com/product-reviews/B01N9YO1DS,227,215.00,399.99,[[i]]
14739,B0731HBTZ7,NRL1029,5,"April 1, 2019",True,Very Satisfied,much to my surprise this phone came completely...,2.0,Apple,"Apple iPhone 7 32GB, Rose Gold (Renewed)",https://www.amazon.com/Apple-iPhone-32GB-Rose-...,https://m.media-amazon.com/images/I/51cRE43zKw...,3.8,https://www.amazon.com/product-reviews/B0731HBTZ7,452,199.95,210.00,[[m]]
12297,B06XRJQX91,Insiya Hasan,3,"April 1, 2018",True,Three Stars,the phone did have scratches and a dent on it ...,2.0,Apple,"Apple iPhone 6S, 64GB, Rose Gold - For AT&T / ...",https://www.amazon.com/Apple-iPhone-Unlocked-D...,https://m.media-amazon.com/images/I/61+mrwyL24...,3.6,https://www.amazon.com/product-reviews/B06XRJQX91,427,158.88,271.84,[[t]]
